In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets evaluate
!conda install protobuf
!pip install accelerate -U
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.8 MB/s eta 

In [3]:
import pdfplumber
import re

def extract_information_from_cv(pdf_path):
    def extract_section(text, section_pattern):
        section_match = section_pattern.search(text)
        section_data = section_match.group(1) if section_match else None
        return section_data

    with pdfplumber.open(pdf_path) as pdf:
        cv_text = ""
        for page in pdf.pages:
            cv_text += page.extract_text()

    summary_pattern = re.compile(r'Summary\n(.+?)\n', re.DOTALL)
    experience_pattern = re.compile(r'Experience\n(.+?)\nEducation', re.DOTALL)
    education_pattern = re.compile(r'Education\n(.+?)\nSkills', re.DOTALL)
    skills_pattern = re.compile(r'Skills\n(.+)', re.DOTALL)

    summary = extract_section(cv_text, summary_pattern)
    experience = extract_section(cv_text, experience_pattern) or "experience"
    education = extract_section(cv_text, education_pattern) or "education"
    skills = extract_section(cv_text, skills_pattern)

    data = f"Summary:\n{summary}\n\nExperience:\n{experience}\n\nEducation:\n{education}\n\nSkills:\n{skills}"
    return data


In [4]:
import csv

data = [
    ["domain", "about"],
]

filename = "/content/drive/MyDrive/NLP/res.csv"
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)
print(f"CSV file '{filename}' created successfully.")


CSV file '/content/drive/MyDrive/NLP/res.csv' created successfully.


In [6]:
import os
parent_path = "/content/drive/MyDrive/NLP/archive__/data/data"
subdirectory = os.listdir(parent_path)
row_data = []
for dir in subdirectory:
	dir_name = dir
	print(dir_name)
	path_of_dir = os.path.join(parent_path,dir)
	list_of_cv = os.listdir(path_of_dir)
	for cv in list_of_cv:
		cv_path = os.path.join(path_of_dir,cv)
		data = extract_information_from_cv(cv_path)
		data_list = [dir_name, data]
		row_data.append(data_list)
	print(f"Finish {dir_name} domain")

AGRICULTURE
Finish AGRICULTURE domain
ARTS
Finish ARTS domain
BPO
Finish BPO domain
APPAREL
Finish APPAREL domain
AVIATION
Finish AVIATION domain
ACCOUNTANT
Finish ACCOUNTANT domain
AUTOMOBILE
Finish AUTOMOBILE domain
BUSINESS-DEVELOPMENT
Finish BUSINESS-DEVELOPMENT domain
ADVOCATE
Finish ADVOCATE domain
BANKING
Finish BANKING domain
HEALTHCARE
Finish HEALTHCARE domain
HR
Finish HR domain
FITNESS
Finish FITNESS domain
FINANCE
Finish FINANCE domain
ENGINEERING
Finish ENGINEERING domain
CONSTRUCTION
Finish CONSTRUCTION domain
DIGITAL-MEDIA
Finish DIGITAL-MEDIA domain
DESIGNER
Finish DESIGNER domain
CHEF
Finish CHEF domain
CONSULTANT
Finish CONSULTANT domain
SALES
Finish SALES domain
PUBLIC-RELATIONS
Finish PUBLIC-RELATIONS domain
TEACHER
Finish TEACHER domain
INFORMATION-TECHNOLOGY
Finish INFORMATION-TECHNOLOGY domain


In [7]:
import csv
filename = "/content/drive/MyDrive/NLP/res.csv"
with open(filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_data)
print("New data appended to the CSV file.")

New data appended to the CSV file.


In [8]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline
import evaluate

In [9]:
md_data = pd.read_csv("/content/drive/MyDrive/NLP/res.csv",names=['labels','about'],skiprows=[0])
df = md_data.head(len(md_data))
df.drop(index=0)
len(df)

2484

In [11]:
df['labels'] = df['labels'].replace(['ACCOUNTANT',
 'ADVOCATE',
 'AGRICULTURE',
 'APPAREL',
 'ARTS',
 'AUTOMOBILE',
 'AVIATION',
 'BANKING',
 'BPO',
 'BUSINESS-DEVELOPMENT',
 'CHEF',
 'CONSTRUCTION',
 'CONSULTANT',
 'DESIGNER',
 'DIGITAL-MEDIA',
 'ENGINEERING',
 'FINANCE',
 'FITNESS',
 'HEALTHCARE',
 'HR',
 'INFORMATION-TECHNOLOGY',
 'PUBLIC-RELATIONS',
 'SALES',
 'TEACHER'],['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
)
print(df['labels'].value_counts())
df.to_csv("res.csv", index=False)

20    120
9     120
10    118
15    118
16    118
0     118
1     118
17    117
6     117
22    116
7     115
18    115
12    115
11    112
21    111
19    110
13    107
4     103
23    102
3      97
14     96
2      63
5      36
8      22
Name: labels, dtype: int64


In [12]:
md_data = pd.read_csv("/content/res.csv",names=['labels','about'],skiprows=[0])

In [13]:
train_data = md_data.sample(frac=0.8, random_state=42)

test_data = md_data.drop(train_data.index)

print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 1987 records.
The testing dataset has 497 records.


In [14]:
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_dataset(data):
    return tokenizer(data["about"],
                     truncation=True,
                     padding="max_length")

dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

print(dataset_train)
print(dataset_test)

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'about', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1987
})
Dataset({
    features: ['labels', 'about', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 497
})


In [16]:
sentiment_classes = [
    'ACCOUNTANT', 'ADVOCATE', 'AGRICULTURE', 'APPAREL', 'ARTS', 'AUTOMOBILE',
    'AVIATION', 'BANKING', 'BPO', 'BUSINESS-DEVELOPMENT', 'CHEF', 'CONSTRUCTION',
    'CONSULTANT', 'DESIGNER', 'DIGITAL-MEDIA', 'ENGINEERING', 'FINANCE', 'FITNESS',
    'HEALTHCARE', 'HR', 'INFORMATION-TECHNOLOGY', 'PUBLIC-RELATIONS', 'SALES', 'TEACHER'
]

id2label = {i: label for i, label in enumerate(sentiment_classes)}
label2id = {label: i for i, label in enumerate(sentiment_classes)}

print("ID to Label Mapping:")
print(id2label)
print("\nLabel to ID Mapping:")
print(label2id)

ID to Label Mapping:
{0: 'ACCOUNTANT', 1: 'ADVOCATE', 2: 'AGRICULTURE', 3: 'APPAREL', 4: 'ARTS', 5: 'AUTOMOBILE', 6: 'AVIATION', 7: 'BANKING', 8: 'BPO', 9: 'BUSINESS-DEVELOPMENT', 10: 'CHEF', 11: 'CONSTRUCTION', 12: 'CONSULTANT', 13: 'DESIGNER', 14: 'DIGITAL-MEDIA', 15: 'ENGINEERING', 16: 'FINANCE', 17: 'FITNESS', 18: 'HEALTHCARE', 19: 'HR', 20: 'INFORMATION-TECHNOLOGY', 21: 'PUBLIC-RELATIONS', 22: 'SALES', 23: 'TEACHER'}

Label to ID Mapping:
{'ACCOUNTANT': 0, 'ADVOCATE': 1, 'AGRICULTURE': 2, 'APPAREL': 3, 'ARTS': 4, 'AUTOMOBILE': 5, 'AVIATION': 6, 'BANKING': 7, 'BPO': 8, 'BUSINESS-DEVELOPMENT': 9, 'CHEF': 10, 'CONSTRUCTION': 11, 'CONSULTANT': 12, 'DESIGNER': 13, 'DIGITAL-MEDIA': 14, 'ENGINEERING': 15, 'FINANCE': 16, 'FITNESS': 17, 'HEALTHCARE': 18, 'HR': 19, 'INFORMATION-TECHNOLOGY': 20, 'PUBLIC-RELATIONS': 21, 'SALES': 22, 'TEACHER': 23}


In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [27]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 40
num_epochs = 30
batches_per_epoch = len(dataset_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [28]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=24, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [30]:
tf_train_set = model.prepare_tf_dataset(
    dataset_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    dataset_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [31]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.compile(optimizer=optimizer)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [32]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [34]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="resume_model",
    tokenizer=tokenizer,
)

/content/resume_model is already a clone of https://huggingface.co/udayGay/resume_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [35]:
callbacks = [metric_callback, push_to_hub_callback]

In [36]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

Epoch 1/30
124/124 [==============================] - ETA: 0s - loss: 3.1654

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


124/124 [==============================] - 225s 1s/step - loss: 3.1654 - val_loss: 3.1440 - accuracy: 0.0402 - precision: 0.0017 - recall: 0.0417 - f1: 0.0033
Epoch 2/30
124/124 [==============================] - ETA: 0s - loss: 3.1553

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


124/124 [==============================] - 172s 1s/step - loss: 3.1553 - val_loss: 3.1474 - accuracy: 0.0282 - precision: 0.0012 - recall: 0.0417 - f1: 0.0023
Epoch 3/30
124/124 [==============================] - ETA: 0s - loss: 3.1208

...

124/124 [==============================] - 138s 1s/step - loss: 1.1101 - val_loss: 1.4298 - accuracy: 0.6640 - precision: 0.5589 - recall: 0.5938 - f1: 0.5692
Epoch 29/30
124/124 [==============================] - ETA: 0s - loss: 1.1134

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


124/124 [==============================] - 140s 1s/step - loss: 1.1134 - val_loss: 1.4298 - accuracy: 0.6640 - precision: 0.5589 - recall: 0.5938 - f1: 0.5692
Epoch 30/30
124/124 [==============================] - ETA: 0s - loss: 1.1040

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


124/124 [==============================] - 140s 1s/step - loss: 1.1040 - val_loss: 1.4298 - accuracy: 0.6640 - precision: 0.5589 - recall: 0.5938 - f1: 0.5692


In [40]:
model.save_pretrained("/content/drive/MyDrive/NLP/archive__/saved_model2")
